In [1]:
%pip install -e git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction

Obtaining MI_prediction from git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction
  Updating ./src/mi-prediction clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 8c3180a37b3921756d0380ef129792414d64fdc4
  Preparing metadata (setup.py) ... done
  Attempting uninstall: MI_prediction
    Found existing installation: MI-prediction 0.1
    Uninstalling MI-prediction-0.1:
      Successfully uninstalled MI-prediction-0.1
  Running setup.py develop for MI_prediction
Note: you may need to restart the kernel to use updated packages.


In [24]:
from moabb.datasets import Cho2017
import mne

In [40]:
ds = Cho2017()

TypeError: __init__() got an unexpected keyword argument 'events'

In [33]:
ds

moabb.datasets.gigadb.Cho2017

In [50]:
data = ds.get_data([1,2])

Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present
Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present


In [51]:
data

{1: {'session_0': {'run_0': <RawArray | 69 x 717300 (1401.0 s), ~377.7 MB, data loaded>}},
 2: {'session_0': {'run_0': <RawArray | 69 x 717300 (1401.0 s), ~377.7 MB, data loaded>}}}

In [39]:
data[1]['session_0']['run_0']

<RawArray | 69 x 717300 (1401.0 s), ~377.7 MB, data loaded>

In [25]:
for subj_id, subj_data in data.items():
     for sess_id, sess_data in subj_data.items():
            for run_id, raw in sess_data.items():
                events = mne.find_events(raw)
                print(events)

200 events found
Event IDs: [1 2]
[[  1023      0      1]
 [  4607      0      1]
 [  8191      0      1]
 [ 11775      0      1]
 [ 15359      0      1]
 [ 18943      0      1]
 [ 22527      0      1]
 [ 26111      0      1]
 [ 29695      0      1]
 [ 33279      0      1]
 [ 36863      0      1]
 [ 40447      0      1]
 [ 44031      0      1]
 [ 47615      0      1]
 [ 51199      0      1]
 [ 54783      0      1]
 [ 58367      0      1]
 [ 61951      0      1]
 [ 65535      0      1]
 [ 69119      0      1]
 [ 72703      0      1]
 [ 76287      0      1]
 [ 79871      0      1]
 [ 83455      0      1]
 [ 87039      0      1]
 [ 90623      0      1]
 [ 94207      0      1]
 [ 97791      0      1]
 [101375      0      1]
 [104959      0      1]
 [108543      0      1]
 [112127      0      1]
 [115711      0      1]
 [119295      0      1]
 [122879      0      1]
 [126463      0      1]
 [130047      0      1]
 [133631      0      1]
 [137215      0      1]
 [140799      0      1]
 [1443

In [26]:
from scipy.io import loadmat

In [41]:
d_mat = loadmat('/Users/julian/mne_data/MNE-gigadb-data/gigadb/pub/10.5524/100001_101000/100295/mat_data/s01.mat',
            squeeze_me=True,
            struct_as_record=False,
            verify_compressed_data_integrity=False,
        )["eeg"]

In [59]:
import matplotlib.pyplot as plt

Fontconfig warning: ignoring UTF-8: not a valid region tag


In [67]:
d_mat.rest.shape[-1]/d_mat.srate

66.5

# Create Cho2017 resting state class

In [128]:
import logging

import numpy as np
from mne import create_info
from mne.channels import make_standard_montage
from mne.io import RawArray
from scipy.io import loadmat
from moabb.datasets import download as dl
from moabb.datasets.base import BaseDataset
from moabb.datasets.base import *
from braindecode.preprocessing.preprocess import exponential_moving_standardize, preprocess, Preprocessor, scale
from braindecode.datasets import create_from_mne_raw
import pandas as pd


In [53]:
log = logging.getLogger(__name__)
GIGA_URL = "ftp://parrot.genomics.cn/gigadb/pub/10.5524/100001_101000/100295/mat_data/"

In [122]:
class Cho2017_Rest(BaseDataset):
    def __init__(self):
        super().__init__(
            subjects=list(range(1, 53)),
            sessions_per_subject=1,
            events=dict(rest=1),
            code="Cho2017_Rest",
            interval=[0, 60],  # full trial is 0-3s, but edge effects
            paradigm="imagery",
            doi="10.5524/100295",
        )

        for ii in [32, 46, 49]:
            self.subject_list.remove(ii)

    def _get_single_subject_data(self, subject):
        """return data for a single subject"""
        fname = self.data_path(subject)

        data = loadmat(
            fname,
            squeeze_me=True,
            struct_as_record=False,
            verify_compressed_data_integrity=False,
        )["eeg"]

        # fmt: off
        eeg_ch_names = [
            "Fp1", "AF7", "AF3", "F1", "F3", "F5", "F7", "FT7", "FC5", "FC3", "FC1",
            "C1", "C3", "C5", "T7", "TP7", "CP5", "CP3", "CP1", "P1", "P3", "P5", "P7",
            "P9", "PO7", "PO3", "O1", "Iz", "Oz", "POz", "Pz", "CPz", "Fpz", "Fp2",
            "AF8", "AF4", "AFz", "Fz", "F2", "F4", "F6", "F8", "FT8", "FC6", "FC4",
            "FC2", "FCz", "Cz", "C2", "C4", "C6", "T8", "TP8", "CP6", "CP4", "CP2",
            "P2", "P4", "P6", "P8", "P10", "PO8", "PO4", "O2",
        ]
        # fmt: on
        emg_ch_names = ["EMG1", "EMG2", "EMG3", "EMG4"]
        ch_names = eeg_ch_names + emg_ch_names 
        ch_types = ["eeg"] * 64 + ["emg"] * 4 
        montage = make_standard_montage("standard_1005")
        resting = data.rest - data.rest.mean(axis=1, keepdims=True)
        
        eeg_rest = resting * 1e-6

        info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=data.srate)
        raw = RawArray(data=eeg_rest, info=info, verbose=False)
        raw.set_montage(montage)

        return {"session_0": {"run_0": raw}}

    def data_path(
        self, subject, path=None, force_update=False, update_path=None, verbose=None
    ):
        if subject not in self.subject_list:
            raise (ValueError("Invalid subject number"))

        url = "{:s}s{:02d}.mat".format(GIGA_URL, subject)
        return dl.data_dl(url, "GIGADB", path, force_update, verbose)

In [123]:
ds_r = Cho2017_Rest()

In [112]:
s1 = ds_r.get_data([1])

In [113]:
s1

{1: {'session_0': {'run_0': <RawArray | 68 x 34048 (66.5 s), ~17.8 MB, data loaded>}}}

In [85]:
def _fetch_and_unpack_moabb_data(dataset, subject_ids):
    data = dataset.get_data(subject_ids)
    raws, subject_ids, session_ids, run_ids = [], [], [], []
    for subj_id, subj_data in data.items():
        for sess_id, sess_data in subj_data.items():
            for run_id, raw in sess_data.items():
                raws.append(raw)
                subject_ids.append(subj_id)
                session_ids.append(sess_id)
                run_ids.append(run_id)
    description = pd.DataFrame({
        'subject': subject_ids,
        'session': session_ids,
        'run': run_ids
    })
    return raws, description

In [ ]:
raws,description = _fetch_and_unpack_moabb_data(ds_r, [1])

In [125]:
raws

[<RawArray | 68 x 34048 (66.5 s), ~17.8 MB, data loaded>]

In [111]:
events = mne.find_events(raws[0])

ValueError: No stim channels found. Consider specifying them manually using the 'stim_channel' parameter.

In [124]:
BaseDataset(raws[0], description)

TypeError: Can't instantiate abstract class BaseDataset with abstract methods _get_single_subject_data, data_path

In [118]:
all_base_ds = [BaseDataset(raw, row)
                       for raw, (_, row) in zip(raws, description.iterrows())]

TypeError: Can't instantiate abstract class BaseDataset with abstract methods _get_single_subject_data, data_path

In [114]:
rest_br = create_from_mne_raw(raws[0],0,0,len(raws[0]),0,False)

ValueError: All picks must be < n_channels (68), got 68

In [ ]:
raws

[<RawArray | 68 x 34048 (66.5 s), ~17.8 MB, data loaded>]